# RAG гарааны код

Энэхүү гарааны jupyter notebook нь энгийн RAG архитектурыг [LangChain](https://python.langchain.com/docs/introduction/) framework-н тусламжтай, хэлний загваруудыг [Hugging Face Transformers](https://huggingface.co/docs/transformers/index)-р татаж, ажиллуулах байдлаар хэрэгжүүлсэн бөгөөд энэхүү notebook-д суурилан өөрсдийн RAG архитектур, загвар, шийдлээ хэрэгжүүлж болно.

Шаардлагатай python сангуудыг [requirements.txt](../requirements.txt) файлд жагсаасан байгаа.

![](https://python.langchain.com/assets/images/rag_concepts-4499b260d1053838a3e361fb54f376ec.png)

In [28]:
import torch
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## Vector Store үүсгэх

RAG системийн үндсэн хоёр хэсгийн нэг, заавал бэлдсэн байх ёстой зүйл бол өгөгдлөө embedding буюу тоон хэлбэрт шилжүүлж, vector store-д хадгалсан байх юм.

![](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*vAvDBIbr8MnL_Q51mBtBhw.png)

### Өгөгдөл унших

`data` хавтсанд байрлах `csv` файлыг уншиж LangChain [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html)-н жагсаалт болгоно.

- `khas_bank_news.csv`
- `khas_bank_pages.csv`
- `khas_bank_products.csv`

In [29]:
DATA_DIR = "../data/"
loader = CSVLoader(file_path=f"{DATA_DIR}khas_bank_news.csv",
                   source_column="link",
                   metadata_columns=["title", "date", "link"],
                   content_columns=["content"],
                   csv_args={
                       "fieldnames": ["title", "link", "date", "content"],
                       }
                   )
news_data = loader.load()[1:]

loader = CSVLoader(file_path=f"{DATA_DIR}khas_bank_products.csv",
                   source_column="link",
                   metadata_columns=["id", "side_menu_text", "link"],
                   content_columns=["content"],
                   csv_args={
                       "fieldnames": ["id", "content", "side_menu_text", "link"],
                       }
                   )
product_data = loader.load()[1:]

loader = CSVLoader(file_path=f"{DATA_DIR}khas_bank_pages.csv",
                   source_column="link",
                   metadata_columns=["title", "link"],
                   content_columns=["content"],
                   csv_args={
                       "fieldnames": ["title", "content", "link"],
                       }
                   )
pages_data = loader.load()[1:]

all_data = news_data + product_data + pages_data
print(len(news_data), len(product_data), len(pages_data))
print(f"Нийт хуудасны тоо: {len(all_data)}")
print(all_data[0])

### Өгөгдөл хуваах

Document хэт урт бол vector store-с хайх хайлтын чанар муудах магадлалтай тул доорх хэсэгт агуулгыг тэмдэгтийн тоогоор таслан хэд хэдэн Document болгов.

Жич: RAG-н чанарыг сайжруулахын тулд зөвхөн тэмдэгтийн тоогоор хуваахаас гадна өгүүлбэр, цогцолбор, бүлэг гэх мэтчилэн өгөгдөлд тохирсон арга олох нь чухал.

In [30]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100
)
docs = text_splitter.split_documents(documents=all_data)

### Embedding

Embedding нь энгийн үгээр ямар нэгэн өгөгдлийг вектор дүрслэл рүү хөрвүүлсэн хэлбэр бөгөөд тухайн вектор нь олон хэмжээст огторгуйд ижил, төстэй утга бүхий өгөгдөлтэй ойр, эсрэг утгатай өгөгдлөөс хол байрладаг.

Энэхүү жишээ RAG-д [`sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2`](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) загварыг сонгон авлаа.

![embedding model](https://python.langchain.com/assets/images/embeddings_concept-975a9aaba52de05b457a1aeff9a7393a.png)

In [31]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

### Vector Store

Харин vector store нь embedding болон өгөгдөл эсвэл түүний заагчийг хамтад нь хадгалж, хайлтын query орж ирэхэд мөн түүнийг вектор хэлбэрт хөрвүүлэн, санд байх векторуудаас хамгийн ойрыг хайдаг өгөгдлийн сан юм.

Энэхүү жишээнд FAISS vector store сонгож ашиглалаа.

![vectorestore](https://python.langchain.com/assets/images/vectorstores-2540b4bc355b966c99b0f02cfdddb273.png)

In [32]:
# Бүх document-с embedding үүсгэж, vector store-д хадгалах
# (өмнө нь үүсгэсэн бол заавал дахин үүсгэх шаардлагагүй, доорх load_local функцээр хадгалсан файлаас уншиж болно)
vector_store = FAISS.from_documents(docs, embeddings)
vector_store.save_local("faiss_index")

In [33]:
# Үүсгэсэн vector store-г файлаас унших
vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

Vector Store үүсгэсэн бол түүнээс ойролцоо утгатай өгөгдөл хайж үзье.

![retriever](https://python.langchain.com/assets/images/retriever_concept-1093f15a8f63ddb90bd23decbd249ea5.png)

In [34]:
# vector store-с document хайгч, k параметраар хамгийн ойр утгатай хэдэн document буцаахыг дамжуулна
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

docs = retriever.invoke("хасбанк хэзээ IPO хийсэн бэ")
docs

[Document(metadata={'source': 'https://www.xacbank.mn/article/ipo-opening', 'row': 73, 'title': 'ХасБанкны IPO: Нээлтийн арга хэмжээ амжилттай боллоо', 'date': '2023.05.25', 'link': 'https://www.xacbank.mn/article/ipo-opening'}, page_content='content: ХасБанкны IPO: Нээлтийн арга хэмжээ амжилттай боллооТус арга хэмжээнд ХасБанк өөрийн үйл ажиллагаа, хувьцаа, ирээдүйн стратеги, зорилтынхоо талаар харилцагч, хөрөнгө оруулагч, олон нийтэд дэлгэрэнгүй мэдээлэл өгсний зэрэгцээ ХасБанкны IPO-д хамтран ажиллаж буй андеррайтерууд болох Райнос Инвестмент үнэт цаасны компани, Өлзий энд Ко Капитал үнэт цаасны компаниуд үнэт цаасны данс нээн, урьдчилсан захиалга авч, хөрөнгийн зах зээлтэй холбоотой зөвлөгөө мэдээлэл өглөө.ХасБанк нийт хувьцааныхаа 5.01 хувийг 677 төгрөгийн нэгжийн үнээр анхдагч зах зээлд арилжаалж 35.7 тэрбум төгрөгийг татан төвлөрүүлэх бөгөөд хувьцааны захиалга 2023 оны 5-р сарын 29-ний 10:00 цагаас 6-р сарын 5-ны 16:00 цаг хүртэл явагдана.Эрхэм харилцагч, хөрөнгө оруулагч та өнө

## Өгөгдөл хайх, хариулах

Өмнөх хэсэгт өгөгдлөө бэлтгэж, vector store-т хадгалсан бол энэ хэсэгт хэрэглэгчийн асуусан асуултаар vector store-с хайлт хийж, LLM-р хариулт өгөх болно.

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*8dUBh25peTpsucMpL1ypUg.png)

### LLM

Хайж олсон мэдээлэл, хэрэглэгчийн асуултад тохирох хариулт үүсгэхийн тулд бичвэр үүсгэгч Large Language Model ашиглана.

Энд Meta компанийн сургасан [8 тэрбум параметртай Llama 3.1(https://huggingface.co/meta-llama/Llama-3.1-8B)] загварыг сонгов. (Hugging Face дээр загвар ашиглах зөвшөөрөл хүссэн маягт бөглөх ёстой)


In [44]:
# Hugging Face-с хандах эрх авсан хаалттай загваруудад хандахын тулд нэвтэрсэн байх шаардлагатай
# Нэмэлт мэдээллийг https://huggingface.co/docs/huggingface_hub/package_reference/authentication
from huggingface_hub import notebook_login

notebook_login()

In [43]:
# Жишээ болгож Llama 3.1 8B загварыг ашиглав
model_id = "meta-llama/Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.float16,
    device_map='auto'
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # llm-н үүсгэж болох токений дээд хязгаар
    max_new_tokens=1024,
    # хариултын randomization-г арилгах
    do_sample=True,
    top_k=1,
    repetition_penalty=1.15,
    # гаралт бидний өгсөн prompt-г хамт хэвлэхгүй байх
    return_full_text=False,
    pad_token_id=tokenizer.eos_token_id,
)

# HuggingFace pipeline-г LangChain-ы pipeline болгох
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

### Chain: Retrieval -> Prompt -> LLM

Prompt-н `{context}`-н оронд хайж олсон мэдээлэл, `{input}`-н оронд хэрэглэгчийн асуусан асуулт орох болно.

In [45]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the user question. If you don't know the answer to the question, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    'Context:\n"""\n{context}\n"""'
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "Question: \"\"\"{input}\"\"\""),
        ("assistant", "Answer: "),
    ]
)

# vector store-с document хайгч, k параметраар хамгийн ойр утгатай хэдэн document буцаахыг дамжуулна
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Chain үүсгэх
# input -> retrieval -> prompt -> llm -> answer
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
# Chain ажиллуулах
response = rag_chain.invoke({"input": "дебит карт яаж авах вэ"})
response["answer"]

In [ ]:
# input, context, answer гурвыг бүгдийг нь харах
response